In [ ]:
# Load modules
from ipyleaflet import (
    Map,
    GeoJSON,
    DrawControl,
    basemaps
)
import datetime as dt
import datacube
import ogr
import matplotlib as mpl
import matplotlib.pyplot as plt
import rasterio
import xarray as xr
from IPython.display import display
import warnings
import ipywidgets as widgets

# Load utility functions
from utils.DEADataHandling import load_clearsentinel2
from utils.utils import transform_from_wgs_poly
from utils.BandIndices import calculate_indices

In [ ]:
dc = datacube.Datacube(app='agriculture-app')

In [ ]:
latitude = (-34.426512, -34.434517)
longitude = (116.648123, 116.630731)
time = ("2015-01-01", "2018-12-31")

In [ ]:
dataset_fc = dc.load(
    product="ls8_fc_albers", latitude=latitude, longitude=longitude, time=time
)

dataset_wofs = dc.load(
    product="wofs_albers", latitude=latitude, longitude=longitude, time=time
)

In [ ]:
shared_times = np.intersect1d(dataset_fc.time, dataset_wofs.time)

ds_fc_matched = dataset_fc.sel(time=shared_times)
ds_wofs_matched = dataset_wofs.sel(time=shared_times)

In [ ]:
dry_mask = masking.make_mask(ds_wofs_matched, dry=True)

In [ ]:
ds_fc_masked = ds_fc_matched.where(dry_mask.water == True)
ds_wofs_masked = masking.make_mask(ds_wofs_matched, wet=True)

In [ ]:
scene=3

plt.figure(figsize=(16,16))
gs = gridspec.GridSpec(3,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
ds_fc_masked.PV.isel(time=scene).plot(cmap='gist_earth_r')
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
ds_fc_masked.BS.isel(time=scene).plot(cmap='Oranges')
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
ds_fc_masked.NPV.isel(time=scene).plot(cmap='copper')
ax3.set_title('NPV')

ax4=plt.subplot(gs[1,1])
ds_fc_masked.UE.isel(time=scene).plot(cmap='magma')
ax4.set_title('UE')

ax5=plt.subplot(gs[2,0])
ds_wofs_masked.water.isel(time=scene).plot(cmap='RdBu')
ax5.set_title('Water')

plt.tight_layout()
plt.show()

In [ ]:
# Suppress warnings
warnings.filterwarnings('ignore')

# Update plotting functionality through rcParams
mpl.rcParams.update({'figure.autolayout': True})

# Define the bounding box that will be overlayed on the interactive map
# The bounds are hard-coded to match those from the loaded data
geom_obj = {
    "type": "Feature",
    "properties": {
        "style": {
            "stroke": True,
            "color": 'red',
            "weight": 4,
            "opacity": 0.8,
            "fill": True,
            "fillColor": False,
            "fillOpacity": 0,
            "showArea": True,
            "clickable": True
        }
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [116.630731, -34.434517],
                [116.630731, -34.426512],
                [116.648123, -34.426512],
                [116.648123, -34.434517],
                [116.630731, -34.434517]
            ]
        ]
    }
}
    
# Create a map geometry from the geom_obj dictionary
# center specifies where the background map view should focus on
# zoom specifies how zoomed in the background map should be
loadeddata_geometry = ogr.CreateGeometryFromJson(str(geom_obj['geometry']))
loadeddata_center = [
    loadeddata_geometry.Centroid().GetY(),
    loadeddata_geometry.Centroid().GetX()
]
loadeddata_zoom = 15

# define the study area map
studyarea_map = Map(
    center=loadeddata_center,
    zoom=loadeddata_zoom,
    basemap=basemaps.Esri.WorldImagery
)

# define the drawing controls
studyarea_drawctrl = DrawControl(
    polygon={"shapeOptions": {"fillOpacity": 0}}
)

# add drawing controls and data bound geometry to the map
studyarea_map.add_control(studyarea_drawctrl)
studyarea_map.add_layer(GeoJSON(data=geom_obj))

# Index to count drawn polygons
global polygon_number
polygon_number = 0

# Define widgets to interact with
instruction = widgets.Output(layout={'border': '1px solid black'})
with instruction:
    print("Draw a polygon within the red box to view a plot of "
          "average NDVI over time in that area.")

info = widgets.Output(layout={'border': '1px solid black'})
with info:
    print("Plot status:")
    
# plot the map
display(instruction)
display(studyarea_map)
display(info)


In [ ]:
ds_fc_masked

In [ ]:
ds_wofs_masked['water_fraction'] = ds_wofs_masked.water.astype(float)
print(ds_wofs_masked)

In [ ]:
ds_combined = xr.merge([ds_fc_masked/100, ds_wofs_masked.water_fraction])

In [ ]:
print(ds_combined)

In [ ]:
ds_combined_resampled = ds_combined.resample(time="2M").mean(skipna=True)

In [ ]:
print(ds_combined_resampled)

In [ ]:
scene=24

plt.figure(figsize=(16,16))
gs = gridspec.GridSpec(3,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
ds_combined_resampled.PV.isel(time=scene).plot(cmap='gist_earth_r')
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
ds_combined_resampled.BS.isel(time=scene).plot(cmap='Oranges')
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
ds_combined_resampled.NPV.isel(time=scene).plot(cmap='copper')
ax3.set_title('NPV')

ax4=plt.subplot(gs[1,1])
ds_combined_resampled.UE.isel(time=scene).plot(cmap='magma')
ax4.set_title('UE')

ax5=plt.subplot(gs[2,0])
ds_combined_resampled.water_fraction.isel(time=scene).plot(cmap='RdBu')
ax5.set_title('Water')

plt.tight_layout()
plt.show()